# Build and Train SiameseNet with Triplet Loss

Now when we have the encodings of all faces, we can train small siamese model (even on our laptop) to distinguish whether 2 images show the same person.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# Path to celeb dataset
PATH = '/workspace/AI/OpenSource/celebrities/'
celeb_data = pd.read_csv(PATH + 'identity_CelebA.txt', sep=" ", header=None)
celeb_data.columns = ["image", "label"]

In [2]:
convfeats = np.load('conv_feats.npy')
labels = celeb_data['label'].values
X_train, X_test, y_train, y_test = train_test_split(convfeats, labels, test_size=0.25)

### Create Siamese Model
Siamese model will encode the conv features to a 256 dim vector that will represent the image. 
Kind of a signature of the images. 

The fundamental assumption of this model is that same person should have similar "signatures". 

In [3]:
from keras.layers import Input, Dense, LeakyReLU, Concatenate, Lambda, BatchNormalization
from keras import backend as K
from keras.models import Model

def embedder(conv_feat_size):
    input = Input((conv_feat_size,), name = 'input')
    normalize = Lambda(lambda x: K.l2_normalize(x, axis=-1), name='normalize')
    x = Dense(512)(input)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dense(128)(x)
    x = normalize(x)
    model = Model(input, x)
    return model
    
def get_siamese_model(conv_feat_size = 1024):
    
    input_a = Input( (conv_feat_size,),  name='anchor')
    input_p = Input( (conv_feat_size,),  name='positive')
    input_n = Input( (conv_feat_size,),  name='negative')
    
    emb_model = embedder(conv_feat_size)
    output_a = emb_model(input_a)
    output_p = emb_model(input_p)
    output_n = emb_model(input_n)
    
    merged_vector = Concatenate(axis=-1)([output_a, output_p, output_n])
    model = Model(inputs=[input_a, input_p, input_n],
                  outputs=merged_vector)

    return model

model = get_siamese_model(conv_feat_size = convfeats.shape[1])

Using TensorFlow backend.


### Create Siamese Model Loss - Triplet Loss

Same person should have 'similar' signatures between his images, whilst new person should have differnet signature.
One way to compare the "similarity" between this signatures (vectors) is to use euclidean distance metric or cosine distance.

I chose to use cosine distance: https://en.wikipedia.org/wiki/Cosine_similarity
But you can easily change it and check if you're getting better results. If you do, let me know :)

Let's define 3 variables:
1. Anchor - The image against which comparisons will be made
2. Positive - Different image of the person in the anchor image
3. Negative - image of a different person

Our loss is essentially: 

###### Loss = Cos_dist(Anchor, Positive) - Cos_dist(Anchor, Negative) + alpha

for more information visit here: https://towardsdatascience.com/siamese-network-triplet-loss-b4ca82c1aec8


In [4]:
def triplet_loss(y_true, y_pred, cosine = True, alpha = 0.2, embedding_size = 128):
    
    ind = int(embedding_size * 2)
    a_pred = y_pred[:, :embedding_size]
    p_pred = y_pred[:, embedding_size:ind]
    n_pred = y_pred[:, ind:]
    if cosine:
        positive_distance = 1 - K.sum((a_pred * p_pred), axis=-1)
        negative_distance = 1 - K.sum((a_pred * n_pred), axis=-1)
    else:
        positive_distance = K.sqrt(K.sum(K.square(a_pred - p_pred), axis=-1))
        negative_distance = K.sqrt(K.sum(K.square(a_pred - n_pred), axis=-1))
    loss = K.maximum(0.0, positive_distance - negative_distance + alpha)
    return loss

### Create image generator for siamese network
The input of the model will be mini-batches of [Anchors, Positives, Negatives] conv features of the images. 

Don't forget - we train on conv features and not on the origianl images

In [5]:
from keras.utils import Sequence

class EmbLoader(Sequence):
    def __init__(self, convfeats, labels, batchSize = 16):
        self.X = convfeats
        self.batchSize = batchSize
        self.y = labels
        self.POS = np.zeros((batchSize, convfeats.shape[1]))
        self.NEG = np.zeros((batchSize, convfeats.shape[1]))
    #gets the number of batches this generator returns
    def __len__(self):
        l,rem = divmod(len(self.y), self.batchSize)
        return (l + (1 if rem > 0 else 0))
    #shuffles data on epoch end
    def on_epoch_end(self):
        a = np.arange(len(self.y))
        np.random.shuffle(a)
        self.X = self.X[a]
        self.y = self.y[a]
    #gets a batch with index = i
    def __getitem__(self, i):
        start = i*self.batchSize
        stop  = (i+1)*self.batchSize
        ancor_labels = self.y[start:stop]
        ancors = self.X[start:stop]
        for k, label in enumerate(ancor_labels):
            pos_idx = np.where(self.y==label)[0]
            neg_idx = np.where(self.y!=label)[0]
            self.NEG[k] = self.X[np.random.choice(neg_idx)]
            pos_idx_hat = pos_idx[(pos_idx<start) | (pos_idx>stop)]
            if len(pos_idx_hat):
                self.POS[k] = self.X[np.random.choice(pos_idx_hat)]
            else:
                # positive examples are within the batch or just 1 example in dataset
                self.POS[k] = self.X[np.random.choice(pos_idx)]
        return [ancors, self.POS[:k+1], self.NEG[:k+1]], np.empty(k+1)

## Launch training

In [6]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
# Compile the model
model.compile(Adam(lr = 0.00005), loss = triplet_loss)


# create generators
train_gen = EmbLoader(X_train, y_train, batchSize = 64)
valid_gen = EmbLoader(X_test, y_test, batchSize = 64)
all_gen = EmbLoader(convfeats, labels, batchSize = 64)

checkpoint = ModelCheckpoint('siamese.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)
model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=150, 
                    validation_data=valid_gen, validation_steps=len(valid_gen),
                    workers=8, use_multiprocessing=True, callbacks=[checkpoint])

# train for 50 more epochs with validation data
model.compile(Adam(lr = 0.00002), loss = triplet_loss)
model.load_weights('siamese.h5')
model.fit_generator(all_gen, steps_per_epoch=len(all_gen), epochs=200, initial_epoch = 150,
                    workers=8, use_multiprocessing=True)
model.save('siamese.h5')

Epoch 1/150
2375/2375 [==============================] - 23s 10ms/step - loss: 0.0710 - val_loss: 0.0612

Epoch 00001: val_loss improved from inf to 0.06117, saving model to siamese.h5
Epoch 2/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0588 - val_loss: 0.0578

Epoch 00002: val_loss improved from 0.06117 to 0.05782, saving model to siamese.h5
Epoch 3/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0556 - val_loss: 0.0553

Epoch 00003: val_loss improved from 0.05782 to 0.05533, saving model to siamese.h5
Epoch 4/150
2375/2375 [==============================] - 22s 9ms/step - loss: 0.0531 - val_loss: 0.0543

Epoch 00004: val_loss improved from 0.05533 to 0.05429, saving model to siamese.h5
Epoch 5/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0521 - val_loss: 0.0527

Epoch 00005: val_loss improved from 0.05429 to 0.05269, saving model to siamese.h5
Epoch 6/150
2375/2375 [==============================] - 21s 9m

2375/2375 [==============================] - 22s 9ms/step - loss: 0.0367 - val_loss: 0.0437

Epoch 00049: val_loss did not improve from 0.04242
Epoch 50/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0367 - val_loss: 0.0435

Epoch 00050: val_loss did not improve from 0.04242
Epoch 51/150
2375/2375 [==============================] - 22s 9ms/step - loss: 0.0367 - val_loss: 0.0428

Epoch 00051: val_loss did not improve from 0.04242
Epoch 52/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0356 - val_loss: 0.0427

Epoch 00052: val_loss did not improve from 0.04242
Epoch 53/150
2375/2375 [==============================] - 22s 9ms/step - loss: 0.0360 - val_loss: 0.0437

Epoch 00053: val_loss did not improve from 0.04242
Epoch 54/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0363 - val_loss: 0.0435

Epoch 00054: val_loss did not improve from 0.04242
Epoch 55/150
2375/2375 [==============================] - 21s 9ms/step 

2375/2375 [==============================] - 21s 9ms/step - loss: 0.0304 - val_loss: 0.0421

Epoch 00100: val_loss did not improve from 0.04154
Epoch 101/150
2375/2375 [==============================] - 22s 9ms/step - loss: 0.0306 - val_loss: 0.0428

Epoch 00101: val_loss did not improve from 0.04154
Epoch 102/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0308 - val_loss: 0.0422

Epoch 00102: val_loss did not improve from 0.04154
Epoch 103/150
2375/2375 [==============================] - 22s 9ms/step - loss: 0.0302 - val_loss: 0.0430

Epoch 00103: val_loss did not improve from 0.04154
Epoch 104/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0305 - val_loss: 0.0423

Epoch 00104: val_loss did not improve from 0.04154
Epoch 105/150
2375/2375 [==============================] - 21s 9ms/step - loss: 0.0307 - val_loss: 0.0428

Epoch 00105: val_loss did not improve from 0.04154
Epoch 106/150
2375/2375 [==============================] - 22s 9ms

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


3166/3166 [==============================] - 26s 8ms/step - loss: 0.0304
Epoch 153/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0302
Epoch 154/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0295
Epoch 155/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0300
Epoch 156/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0299
Epoch 157/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0297
Epoch 158/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0294
Epoch 159/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0293
Epoch 160/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0296
Epoch 161/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0290
Epoch 162/200
3166/3166 [==============================] - 26s 8ms/step - loss: 0.0292
Epoch 163/200
3166/3166 [==============================] 